In [17]:
from stable_baselines3 import PPO
from environment import create_environment
import chess
from expert import ChessExpert, ModelOpponent
from utils import FrequentLoggingCallback, create_chess_agent, selections_to_move

# 1.1 Create the env exactly as during training (masks, opponent off if self‑play)
env = create_environment(
    use_expert=True,
    include_masks=True,
    stockfish_opponent=None,
    agent_color=chess.WHITE,
)
agent = create_chess_agent(
    env,
    gamma=0.99,
    n_steps=512,
    learning_rate=3e-4
)
env.expert_model = ChessExpert()
env.stockfish_opponent = ModelOpponent(agent, include_masks=True)
# 1.2 Load the model
model = PPO.load("trained_models/20250420_140954/final_model.zip", env=env)
# If you didn’t pickle the env, just do: model = PPO.load("ppo_chess.zip")

Created Stockfish opponent with depth 15
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/pd468/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
import os
import numpy as np
import imageio
import chess.svg
from cairosvg import svg2png


def record_episode(model, env, out_dir="frames"):
    # Make output dir
    os.makedirs(out_dir, exist_ok=True)
    obs = env.reset()
    done = False
    frame_paths = []
    step = 0

    while not done:
        # 2.1 Get action (deterministic or stochastic)
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)

        # 2.2 Render board to SVG, convert, save PNG
        board = env.current_board  # the python-chess Board
        svg = chess.svg.board(board=board, size=350)
        png_data = svg2png(bytestring=svg)
        frame_path = os.path.join(out_dir, f"frame_{step:03d}.png")
        with open(frame_path, "wb") as f:
            f.write(png_data)
        frame_paths.append(frame_path)
        step += 1

    return frame_paths


# Example usage:
frames = record_episode(model, env)

In [20]:
imageio.get_writer("chess_game.mp4", fps=2, codec="libx264")

def make_video(frame_paths, output_path="chess_game.mp4", fps=1):
    # For MP4
    writer = imageio.get_writer(output_path, fps=fps, codec="libx264", quality=8)
    for fp in frame_paths:
        img = imageio.imread(fp)
        writer.append_data(img)
    writer.close()


def make_gif(frame_paths, output_path="chess_game.gif", fps=1):
    # For GIF
    images = [imageio.imread(fp) for fp in frame_paths]
    imageio.mimsave(output_path, images, fps=fps)


# Example usage:
make_video(frames, "chess_game.mp4", fps=1)  # 1 frame per second
# or
make_gif(frames, "chess_game.gif", fps=1)

/tmp/ipykernel_555061/3443890228.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = imageio.imread(fp)
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (350, 350) to (352, 352) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


/tmp/ipykernel_555061/3443890228.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(fp) for fp in frame_paths]
